In [0]:
from pyspark.sql.functions import count, col

df = spark.read.format("delta").load(
    "/Volumes/workspace/default/silver/corte_agua/"
)

display(df)


In [0]:
# coluna de bairro
col_bairro = None
for c in df.columns:
    if "bairro" in c:
        col_bairro = c
        break

if col_bairro is None:
    raise Exception("Nenhuma coluna de bairro encontrada!")

In [0]:
# Agrupamento por bairro
df_bairro = (
    df.groupBy(col_bairro)
    .agg(count("*").alias("qtd_cortes"))
    .orderBy(col("qtd_cortes").desc())
)

display(df_bairro)

In [0]:
# Salvar Gold
df_bairro.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/default/gold/corte_agua/cortes_por_bairro/")

print("Gold 02 — Cortes por bairro gerado com sucesso!")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_corte_agua_gold_02 AS
SELECT *
FROM delta.`/Volumes/workspace/default/gold/corte_agua/cortes_por_bairro`;
